### This is example of communication among participants in **DIDCOMM** manner
We have 3 members: **Alice**, **Bob**, **Carl**. 

For demo purposes we maintain Wallets in the cloud<br/>
You can check it by visiting [https://agents.socialsirius.com/](https://agents.socialsirius.com/) <br/>
Login: **didcomm** Password: **didcomm**


In [2]:
import sirius_sdk

# Init connection to Wallet that maintained in the Cloud
# Off cource developer have alternative - he may maintain Wallet and secrets on other way (KERI library) 
# thanks to Sirius-SDK is friendly to Legacy and flexible due to dependency injection architecture 

alice_cfg = sirius_sdk.Config().setup_cloud(
    server_uri='https://agents.socialsirius.com',
    credentials='s7RxiBlpeNq8k8hrx4vlgjb8XFnGQGxTjIQgk74LgYSYAS4TuR1kZZxJg5MS6b+rwsta0b6XT84KQyxKcKkysg==',
    p2p={
        "their_verkey": "9b5fhAxyXoWP3ZHtpvKEFUf8o47uNcysuSd1c1VDq2yo",
        "my_keys": [
          "HYjGQGEtCbLisC2W3eeYxi196ojLV8sSdmGAAHt1Riry",
          "4MFTdbvvUVnNkQpU7UHJXz1Qea9WqvDXgbkxNetoBhCob45VUDS3Ef5fN3nLBqkGNSG5RftK9RidY7HxYcBp38Lm"
        ]
   }
)
 

### Ok, we initialized our environment, Let's establish connections with others
**Alice** will generate *Invitation URL* and share it with **Bob** and **Carl** via mesenger or email...

In [3]:
import json
import logging
import asyncio
from typing import Optional

import ipywidgets as widgets

# 
logging.getLogger().setLevel(logging.CRITICAL)

# Variables to store Peer-To-Peer metadata to communicate
BOB: Optional[sirius_sdk.Pairwise] = None
CARL: Optional[sirius_sdk.Pairwise] = None
CONN_KEY: Optional[str] = None
# Seed to restore self identity
ALICE_SEED = '000000000000000000000000000ALICE'
        

async def generate_invitation():
    try:
        async with sirius_sdk.context(alice_cfg):
            CONN_KEY = await sirius_sdk.Crypto.create_key()
            endpoint = [e for e in await sirius_sdk.endpoints() if e.routing_keys == []][0]
            invitation = sirius_sdk.aries_rfc.Invitation(
                # Название компании
                label='Alice',
                # интернет-адрес P2P соединения
                endpoint=endpoint.address, 
                # публичный ключ ЭЦП компании
                recipient_keys=[CONN_KEY]
            )
            # output.append_stdout('\nInvitation JSON:\n')
            # output.append_stdout(json.dumps(invitation, indent=2, sort_keys=True))
            invitation_url = 'https://alice.com/invitation' + invitation.invitation_url
            output.append_stdout('\n !!!!!! Copy invitation and PASS to Bob and Carl !!!!!!\n')
            output.append_stdout('_'*128 + '\n')
            output.append_stdout(invitation_url)
            output.append_stdout('\n' + '_'*128)
            # Start ti listen connection requests
            output.append_stdout('\nWait for connection requests from others...')
            listener = await sirius_sdk.subscribe()
            async for event in listener:
                msg = event.message
                if isinstance(msg, sirius_sdk.aries_rfc.ConnRequest) and event.recipient_verkey == CONN_KEY:
                    output.append_stdout('\nReceived conn-request, start rfc-160 to establish connection...')
                    did, verkey = await sirius_sdk.DID.create_and_store_my_did(seed=ALICE_SEED)
                    output.append_stdout(f'\nMy DID: {did}  My Verkey: {verkey}  My Endpoint: {endpoint.address}')
                    ########
                    rfc_0160 = sirius_sdk.aries_rfc.Inviter(
                        me=sirius_sdk.Pairwise.Me(did, verkey),
                        connection_key=CONN_KEY,
                        my_endpoint=endpoint
                    )
                    success, p2p = await rfc_0160.create_connection(request=msg)
                    if success:
                        their_label = p2p.their.label
                        output.append_stdout('_'*128 + '\n')
                        output.append_stdout(f'\n!!! Connection with {their_label} was established !!!!')
                        output.append_stdout('\n' + '_'*128)
                        if their_label == 'Carl':
                            CARL = p2p
                        elif their_label == 'Bob':
                            BOB = p2p
                    else:
                        output.append_stderr(rfc_0160.problem_report.explain)
                    ########
    except Exception as e:
        output.append_stderr(repr(e))

output = widgets.Output()
button = widgets.Button(description="Generate Invitation", button_style='primary')

def on_button_clicked(b):
    output.clear_output()
    asyncio.ensure_future(generate_invitation())
button.on_click(on_button_clicked)
        
display(button, output)


Button(button_style='primary', description='Generate Invitation', style=ButtonStyle())

Output()